In [38]:

import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from PIL import Image
from google.colab import files

def load_images_and_labels(uploaded_files):
    images = []
    labels = []

    for filename, file_content in uploaded_files.items():
        if filename.endswith(".png"):
            # Decode the uploaded file content into an image
            image_array = np.frombuffer(file_content, dtype=np.uint8)
            img = cv2.imdecode(image_array, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (128, 32))  # Resize to uniform size
            images.append(img)

            # Assuming labels are encoded in the filename before an underscore '_'
            label = filename.split('_')[0]
            labels.append(label)

    return np.array(images), np.array(labels)




In [39]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

def preprocess_labels(labels):
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    labels_categorical = to_categorical(labels_encoded)
    return labels_categorical, label_encoder

In [40]:
def build_model(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 128, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model


In [41]:
def main():
    # Upload the images using the file upload widget
    uploaded = files.upload()

    images, labels = load_images_and_labels(uploaded)
    images = np.expand_dims(images, -1)  # Add channel dimension

    labels, label_encoder = preprocess_labels(labels)
    num_classes = labels.shape[1]

    # Check if there are enough samples to split
    if len(images) < 2:
        print("Error: Insufficient number of images to perform train-test split.")
        return  # Exit the function early

    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

    model = build_model(num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {test_acc:.4f}")

if __name__ == '__main__':
    main()

Saving CNN Image.png to CNN Image (4).png
Error: Insufficient number of images to perform train-test split.
